# Now let's see how to add timelimit to the environment

After 4 steps the truncated is being set to True, indicating that the episode has ended due to the time limit.

Using a test Model

In [ ]:
from stable_baselines3 import PPO
import gymnasium as gym
import assembly_game
from assembly_game.processor import PROCESSOR_ACTIONS

In [2]:
MAX_STEPS = 30
env = gym.make("Min4Game", max_episode_steps=MAX_STEPS)

In [3]:
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=200000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.9     |
|    ep_rew_mean     | 3.74     |
| time/              |          |
|    fps             | 2656     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 24.1        |
|    ep_rew_mean          | 4.33        |
| time/                   |             |
|    fps                  | 1927        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014059145 |
|    clip_fraction        | 0.126       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [4]:
state, _ = env.reset()
cumreward = 0
for i in range(MAX_STEPS):
  action,_ = model.predict(state)
  state, reward, terminated, truncated, info = env.step(action)
  cumreward +=reward
  print(PROCESSOR_ACTIONS[action], info, reward)
  if terminated or truncated:
    print(terminated)
    print(truncated)
    print(f"Episode finished after {i+1} timestamps")
    break
print(f"total reward {cumreward}")

(<Instruction.MOV: 0>, <Operand.RDI: 0>, <Operand.RAX: 4>) {'example_0': 'rdi=1 rsi=2 rax=1 rdx=3 rcx=4 cmp_res=0', 'example_1': 'rdi=1 rsi=2 rax=1 rdx=4 rcx=3 cmp_res=0', 'example_2': 'rdi=1 rsi=3 rax=1 rdx=2 rcx=4 cmp_res=0', 'example_3': 'rdi=1 rsi=3 rax=1 rdx=4 rcx=2 cmp_res=0', 'example_4': 'rdi=1 rsi=4 rax=1 rdx=2 rcx=3 cmp_res=0', 'example_5': 'rdi=1 rsi=4 rax=1 rdx=3 rcx=2 cmp_res=0', 'example_6': 'rdi=2 rsi=1 rax=2 rdx=3 rcx=4 cmp_res=0', 'example_7': 'rdi=2 rsi=1 rax=2 rdx=4 rcx=3 cmp_res=0', 'example_8': 'rdi=2 rsi=3 rax=2 rdx=1 rcx=4 cmp_res=0', 'example_9': 'rdi=2 rsi=3 rax=2 rdx=4 rcx=1 cmp_res=0', 'example_10': 'rdi=2 rsi=4 rax=2 rdx=1 rcx=3 cmp_res=0', 'example_11': 'rdi=2 rsi=4 rax=2 rdx=3 rcx=1 cmp_res=0', 'example_12': 'rdi=3 rsi=1 rax=3 rdx=2 rcx=4 cmp_res=0', 'example_13': 'rdi=3 rsi=1 rax=3 rdx=4 rcx=2 cmp_res=0', 'example_14': 'rdi=3 rsi=2 rax=3 rdx=1 rcx=4 cmp_res=0', 'example_15': 'rdi=3 rsi=2 rax=3 rdx=4 rcx=1 cmp_res=0', 'example_16': 'rdi=3 rsi=4 rax=3 rdx=1